In [1]:
## Description of Phase II Part 1
# Binary integer program (BIP) that assigns carry-in sites to prep sites
# This model does not consider delivery windows or vehicle capacity, those constraints will be considered when routing

from __future__ import print_function ## Not sure if necessary
from ortools.linear_solver import pywraplp ## Solves the binary assignment
import pandas as pd ## Reads in .csv files

In [2]:
## Helpers
def convert_to_dict(keylist,valuelist):
    #makes two lists into a dictionary with the first list as the keys
    #list lengths must be equivalent
    temp_dict = {}
    if len(keylist) != len(valuelist):
        print("List lengths not equivalent")
        return
    for x in keylist:
        temp_dict[x] = valuelist[keylist.index(x)]
    return temp_dict

In [3]:
## Read in data
## change to personal computer path

# Supply
data = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/ImportantDataCompiled.xlsx", "Prep Site Capacity",usecols = "B,C")
data = data.dropna().astype(int) # drop n/a values and convert to integers
prep_site_number = data["Prep Site #"].values.tolist() ## makes column into list
supply = data["Supply"].values.tolist()

# Demand
demanddata = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/ImportantDataCompiled.xlsx","Meals per School")
carry_in_site_number = demanddata["School #"].values.tolist() 
demanddata["Breakfast Demand"] = demanddata["Breakfast Demand"].astype(int)
demanddata["Lunch Demand"] = demanddata["Lunch Demand"].astype(int)
breakfast_demand = demanddata["Breakfast Demand"].dropna()
lunch_demand = demanddata["Lunch Demand"].dropna()
demand = breakfast_demand + lunch_demand
demand = demand.values.tolist()

In [4]:
cost = [1,2,3,4]

carry_in_range = range(len(demand)) ## means carry_in_range takes values from 0 to len(demand) - 1
prep_range = range(len(supply))

# get information into dictionaries
carry_in_dict = convert_to_dict(carry_in_site_number,demand)
prep_site_dict = convert_to_dict(prep_site_number,supply)

In [5]:
## Model 
# Declare solver       
solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [6]:
## Declare binary decision variables
#i by j array segments where (i,j) is 1 if carry_in i is assigned to prep j and 0 otherwise
# segments is a dictionary which takes the tuple (i,j) as the key and the bool as the value
## CHANGE so the tuple is the prep or carry_in site number??? 
segments = {}
for i in carry_in_range:
    for j in prep_range:
        a = carry_in_site_number[i]
        b = prep_site_number[j]
        #convert i and j to the schools they refer to
        segments[(i, j)] = solver.BoolVar('seg[%i,%i]' % (a, b))

In [ ]:
## Objective: minimize total distance between prep and carry sites 
# does not represent optimal minimization of drive time, but ensures proximity
# Want a cost dictionary that keys on the tuple i,j for the distance/time between i and j where i and j
solver.Minimize(solver.Sum([cost[i][j] * segments[i,j] for i in carry_in_range
                                                       for j in prep_range]))

In [ ]:
## Constraints
# carry_in i is assigned one and only one prep j (prep j can have multiple carry_ins)
# I think this is creating one sum for each carry_in i (do I need to switch i and j?)
for i in carry_in_range:
    solver.Add(solver.Sum([segments[i, j] for j in prep_range]) == 1)

# sum of demand of carry_ins I assigned to j do not exceed supply of prep j
for j in prep_range:
    solver.Add(solver.Sum([segments[i,j]*demand[i]] for i in carry_in_range) <= supply[j])

In [ ]:
## Results
print('Total distance = ', solver.Objective().Value())

#Prints assignments
for i in carry_in_range:
    for j in prep_range:
        if segments[i, j].solution_value() > 0:
            a = carry_in_site_number[i]
            b = prep_site_number[j]
            print('Carry in site %d assigned to prep site %d.' % (a,b))
